### 스타벅스 입점데이터 크롤링

In [32]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)
time.sleep(3)

In [33]:
# '서울' > '전체' 버튼 클릭

seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()
time.sleep(3)

all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()
time.sleep(3)

In [34]:
# HTML 파서 만들기

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [35]:
# html 태그를 이용하여 각 매장 정보 가져오기

starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))

519


In [36]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [37]:
# 매장 정보 샘플 확인
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0][4:]
address = str(starbucks_store.select('p')[0]).split('<br/>')[0].split('>')[1]
tel = str(starbucks_store.select('p')[0]).split('<br/>')[1].split('<')[0]

In [38]:
print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)

역삼아레나빌딩
37.501087
127.043069
general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [39]:
# 서울시 스타벅스 매장 목록 데이터 만들기

starbucks_list = []
for item in starbucks_soup_list:
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0][4:]
    address = str(item.select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p')[0]).split('<br/>')[1].split('<')[0]
    
    starbucks_list.append([name, lat, lng, store_type, address, tel])

In [40]:
# starbucks_list를 데이터프레임으로 생성
columns = ['매장명', '위도', '경도', '매장타입', '주소', '전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [41]:
seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     519 non-null    object
 1   위도      519 non-null    object
 2   경도      519 non-null    object
 3   매장타입    519 non-null    object
 4   주소      519 non-null    object
 5   전화번호    519 non-null    object
dtypes: object(6)
memory usage: 24.5+ KB


In [42]:
# 엑셀로 저장
seoul_starbucks_df.to_excel('crawling_data/4_1_seoul_starbucks_list.xlsx', index=False)

### 공공데이터 수집

In [1]:
import requests
import pandas as pd

# 서울 열린데이터광장 API 호출 키
SEOUL_API_AUTH_KEY = '****************************'

# 시군구 목록을 가져오는 API 호출 URL
url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_AUTH_KEY}/json/SdeTlSccoSigW/1/25/'
print(url)

http://openapi.seoul.go.kr:8088/****************************/json/SdeTlSccoSigW/1/25/


In [44]:
# 서울 열린데이터광장 OPEN API 호출
result_dict = requests.get(url).json()
result_dict

{'SdeTlSccoSigW': {'list_total_count': 25,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'OBJECTID': 1.0,
    'SIG_CD': '11320',
    'SIG_KOR_NM': '도봉구',
    'SIG_ENG_NM': 'Dobong-gu',
    'ESRI_PK': 0.0,
    'LAT': '37.6658609',
    'LNG': '127.0317674'},
   {'OBJECTID': 2.0,
    'SIG_CD': '11380',
    'SIG_KOR_NM': '은평구',
    'SIG_ENG_NM': 'Eunpyeong-gu',
    'ESRI_PK': 1.0,
    'LAT': '37.6176125',
    'LNG': '126.9227004'},
   {'OBJECTID': 3.0,
    'SIG_CD': '11230',
    'SIG_KOR_NM': '동대문구',
    'SIG_ENG_NM': 'Dongdaemun-gu',
    'ESRI_PK': 2.0,
    'LAT': '37.5838012',
    'LNG': '127.0507003'},
   {'OBJECTID': 4.0,
    'SIG_CD': '11590',
    'SIG_KOR_NM': '동작구',
    'SIG_ENG_NM': 'Dongjak-gu',
    'ESRI_PK': 3.0,
    'LAT': '37.4965037',
    'LNG': '126.9443073'},
   {'OBJECTID': 5.0,
    'SIG_CD': '11545',
    'SIG_KOR_NM': '금천구',
    'SIG_ENG_NM': 'Geumcheon-gu',
    'ESRI_PK': 4.0,
    'LAT': '37.4600969',
    'LNG': '126.9001546'},
   {'OBJECTID': 6.0

In [45]:
result_data = result_dict['SdeTlSccoSigW']

print(result_data['list_total_count'])
print(result_data['RESULT'])
print(result_data['row'][0])

25
{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}
{'OBJECTID': 1.0, 'SIG_CD': '11320', 'SIG_KOR_NM': '도봉구', 'SIG_ENG_NM': 'Dobong-gu', 'ESRI_PK': 0.0, 'LAT': '37.6658609', 'LNG': '127.0317674'}


In [46]:
data_list = result_data['row']
sample_df = pd.DataFrame(data_list)
sample_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


#### 서울열린데이터광장 OPEN API를 호출하는 함수 작성

    서울 열린데이터광장 OPEN API는 동일한 구조이므로 함수로 작성하여 활용

In [47]:
# 서울열린데이터광장 API 호출 키

SEOUL_API_AUTH_KEY = '*******************'

# 서울열린데이터광장 OPEN API 호출을 위한 공통 함수
def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list

#### 시군구 목록 데이터 수집후 엑셀 다운로드

In [48]:
# url주소 내 항목들은 서울열린데이터광장 open api탭에서 확인가능
sgg_url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_AUTH_KEY}/json/SdeTlSccoSigW/1/25/'

sgg_data_list = seoul_open_api_data(url, 'SdeTlSccoSigW')
sgg_data_list[0]

{'OBJECTID': 1.0,
 'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0.0,
 'LAT': '37.6658609',
 'LNG': '127.0317674'}

In [49]:
# 결과데이터를 pandas의 데이터프레임으로 변환
columns = ['SIG_CD', 'SIG_KOR_NM', 'LAT', 'LNG']
sgg_df = pd.DataFrame(data = sgg_data_list, columns = columns)
sgg_df.head()

,SIG_CD,SIG_KOR_NM,LAT,LNG
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [50]:
sgg_df.columns = ['시군구코드', '시군구명', '위도', '경도']
sgg_df.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [51]:
sgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   25 non-null     object
 1   시군구명    25 non-null     object
 2   위도      25 non-null     object
 3   경도      25 non-null     object
dtypes: object(4)
memory usage: 928.0+ bytes


In [52]:
# 데이터 엑셀로 저장

sgg_df.to_excel('crawling_data/4_2_seoul_sgg_list.xlsx', index = False)

### 공통함수를 이용 서울시 시군구별 인구데이터 가져오기

In [53]:
pop_url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_AUTH_KEY}/json/octastatapi419/1/26/'
pop_data_list = seoul_open_api_data(pop_url, 'octastatapi419')

sgg_pop_df = pd.DataFrame(data = pop_data_list)

In [54]:
sgg_pop_df.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,﻿2010.1-4,합계,"4,121,193","10,464,171","5,181,117","5,283,054","10,208,670","5,059,504","5,149,166","255,501","121,613","133,888",2.48,"956,414"
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"


In [55]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GIGAN                  26 non-null     object
 1   JACHIGU                26 non-null     object
 2   SEDAE                  26 non-null     object
 3   GYE_1                  26 non-null     object
 4   NAMJA_1                26 non-null     object
 5   YEOJA_1                26 non-null     object
 6   GYE_2                  26 non-null     object
 7   NAMJA_2                26 non-null     object
 8   YEOJA_2                26 non-null     object
 9   GYE_3                  26 non-null     object
 10  NAMJA_3                26 non-null     object
 11  YEOJA_3                26 non-null     object
 12  SEDAEDANGINGU          26 non-null     object
 13  N_65SEISANGGORYEONGJA  26 non-null     object
dtypes: object(14)
memory usage: 3.0+ KB


In [56]:
# 필요없는 합계 데이터 제거
sgg_pop_df_selected = sgg_pop_df[sgg_pop_df['JACHIGU'] != '합계']
sgg_pop_df_selected.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"
5,2010.1-4,광진구,"156,264","386,292","190,823","195,469","373,586","185,087","188,499","12,706","5,736","6,970",2.39,"30,765"


In [57]:
# 분석에 필요한 칼럼만 선택
columns = ['JACHIGU', 'GYE_1']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.columns = ['시군구명', '주민등록인구']
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"


In [58]:
# 데이터를 엑셀파일로 저장
sgg_pop_df_final.to_excel('crawling_data/4_3_sgg_pop.xlsx',index = False)

#### 공통함수를 이용해 서울시 시군구별 사업체 데이터 가져오기

In [59]:
biz_url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_AUTH_KEY}/json/octastatapi104/1/450/'
biz_data_list = seoul_open_api_data(biz_url, 'octastatapi104')

sgg_biz_df = pd.DataFrame(biz_data_list)
sgg_biz_df.head(10)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
0,2015,합계,합계,820658,272567,5108828,2887719,2221109,17,491,...,8713,463,6712,22695,360192,234494,912406,93642,279921,126785
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
2,2015,종로구,사직동,3551,1092,51425,29781,21644,1,16,...,2,2,32,38,4158,563,4074,280,2281,856
3,2015,종로구,삼청동,738,354,4736,2374,2362,-,-,...,-,-,-,6,15,273,864,20,84,261
4,2015,종로구,부암동,540,224,3635,1714,1921,-,-,...,-,-,-,22,53,104,523,33,37,125
5,2015,종로구,평창동,665,298,3059,1438,1621,-,-,...,-,-,-,18,84,129,458,34,123,155
6,2015,종로구,무악동,581,121,1510,804,706,-,-,...,-,-,-,9,79,59,158,344,354,38
7,2015,종로구,교남동,320,103,3158,1342,1816,-,-,...,-,-,-,9,40,69,247,32,72,62
8,2015,종로구,가회동,755,368,12215,8804,3411,-,-,...,-,-,-,11,5446,172,395,23,40,243
9,2015,종로구,종로1.2.3.4가동,16072,5081,110407,64972,45435,-,-,...,383,4,54,88,13977,7627,19952,176,1947,2304


In [60]:
sgg_biz_df_selected = sgg_biz_df[sgg_biz_df['DONG']=='소계']
sgg_biz_df_selected.head(10)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
19,2015,중구,소계,66190,24249,423808,234337,189471,-,-,...,1048,8,159,451,7364,34785,100132,1666,16214,6142
35,2015,용산구,소계,21178,7030,133446,76287,57159,-,-,...,55,12,167,440,7483,7714,30638,1292,4458,4245
52,2015,성동구,소계,26130,7617,162019,97243,64776,-,-,...,816,31,439,661,6049,7090,31229,2641,7832,3494
70,2015,광진구,소계,24531,8627,123689,69279,54410,-,-,...,93,16,145,836,10047,6214,20209,2511,7399,4555
86,2015,동대문구,소계,32994,10813,143858,77571,66287,-,-,...,1,17,134,777,6390,11740,33122,3600,7362,5101
101,2015,중랑구,소계,27287,8952,99241,51968,47273,-,-,...,105,19,208,906,5146,5540,16872,6225,12937,4010
118,2015,성북구,소계,24690,8883,113893,57411,56482,-,-,...,6,16,184,594,3894,5453,16944,3601,6921,4411
139,2015,강북구,소계,18654,6738,69787,33584,36203,-,-,...,206,8,149,481,2018,3976,10676,3131,6539,3667
153,2015,도봉구,소계,18455,6142,68669,36066,32603,1,1,...,-,12,119,493,2832,3667,11499,4786,9675,2675


In [61]:
# 필요없는 칼럼 삭제 및 인덱스 초기화

columns = ['JACHIGU', 'GYE', 'SAEOPCHESU_1']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명', '종사자수', '사업체수']
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop = True)

sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531


In [62]:
# 엑셀로 저장

sgg_biz_df_final.to_excel('crawling_data/4_3_sgg_biz.xlsx', index = False)